# Text Classification Using Transformer Networks (BERT)

Some initialization:

In [1]:
!pip3 install datasets
!pip3 install transformers
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 46.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40

In [2]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


Read the train/dev/test datasets and create a HuggingFace `Dataset` object:

In [3]:
def read_league_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["message","toxicity_label"]]

In [4]:
#labels = open('classes.txt').read().splitlines()
labels=["0","1"]
league_data = read_league_data('dataToxic.csv')
print(labels)
league_data = league_data.rename(columns={"toxicity_label": "label"})
league_data['message'] = league_data['message'].str.replace(',',' ',regex=False)
league_data['label'] = league_data['label'].apply(lambda x: 1 if x == 'toxic' else 0)
league_data

['0', '1']


,message,label
0,report for unskilled player is useless thx <3 ...,0
1,mimimi,0
2,im comming for you riven pfft focus Zed always...,1
3,thx top no flash for what ? he has 2 kill in l...,1
4,IIII ISI K udyr top dnt us see it? CAMP MORE P...,0
...,...,...
88083,gj &gt;&lt; xD i said ss gj stop go alone plz ...,1
88084,i like the new un-do button GET BACK GET BACK ...,1
88085,thx now i can b its ok re top sry thought cait...,0
88086,take t brb swian i was ogin b i was going blue...,1


In [5]:
def read_kaggle_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["Text","oh_label"]]

In [6]:
kaggle_data = read_kaggle_data("kaggle_parsed_dataset.csv")
kaggle_data = kaggle_data.rename(columns={"oh_label": "label", "Text": "message"})
kaggle_data

,message,label
0,"""You fuck your dad.""",1
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0
...,...,...
8794,"""Never really gave it much thought. I just fig...",0
8795,"""Nadie se salva de la regla 34 xd""",0
8796,"""Question: Are you a boy or a girl?""",0
8797,"""Leave your email or phone number and maybe yo...",1


In [7]:
def read_dota_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["text","target"]]

In [8]:
dota_data = read_dota_data('tagged-data.csv')
dota_data = dota_data.rename(columns={"text": "message", "target": "label"})
dota_data['label'] = dota_data['label'].replace(2,1)
dota_data

,message,label
0,COMMEND ME TY,0
1,sorry nex,0
2,what is the best soup?,0
3,man that silence on axe,0
4,not coming into play,0
...,...,...
3262,"wt?f?asfU JGOFIDLK,YH",1
3263,you must really suck,1
3264,YOU HAVE IDIOT PLAYER,1
3265,SUPER IDIOT,1


In [10]:
from sklearn.model_selection import train_test_split

dota_train, dota_test = train_test_split(dota_data, train_size=0.031, random_state = 4)
dota_train.reset_index(inplace=True, drop=True)
dota_test.reset_index(inplace=True, drop=True)

print(f'dota train rows: {len(dota_train.index):,}')
print(f'dota test rows: {len(dota_test.index):,}')

dota train rows: 101
dota test rows: 3,166


In [11]:
data = pd.concat([dota_train,kaggle_data,league_data])
data

,message,label
0,happy new year,0
1,eat dick bitch,1
2,like to play op heroes ench shit?,1
3,fuck off,1
4,funny team,0
...,...,...
88083,gj &gt;&lt; xD i said ss gj stop go alone plz ...,1
88084,i like the new un-do button GET BACK GET BACK ...,1
88085,thx now i can b its ok re top sry thought cait...,0
88086,take t brb swian i was ogin b i was going blue...,1


In [19]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(data, train_size=0.999, random_state = 4)
train_df.reset_index(inplace=True, drop=True)
eval_df.reset_index(inplace=True, drop=True)

print(f'train rows: {len(train_df.index):,}')
print(f'eval rows: {len(eval_df.index):,}')

train rows: 96,891
eval rows: 97


In [20]:
eval_df

,message,label
0,we lose. this game is lost. and udyr babysit t...,1
1,omg team... sorry telephone.. heimer very noob...,1
2,aatrox winning by doing nothin padedigkqQ wow ...,1
3,smitles blue please nope if they take enough a...,1
4,llamas? too late olaf my farm idiots report ol...,1
...,...,...
92,karma mid? gj gj lee gj should have backed.......,1
93,ok i will not push then top no ward top no ward,0
94,ops runes 0-0 me sona op ss ss mid a&nd top pf...,1
95,ggwp,1


In [21]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train_df)
ds['validation'] = Dataset.from_pandas(eval_df)
ds

DatasetDict({
    train: Dataset({
        features: ['message', 'label'],
        num_rows: 96891
    })
    validation: Dataset({
        features: ['message', 'label'],
        num_rows: 97
    })
})

Tokenize the texts:

In [22]:
from transformers import AutoTokenizer

transformer_name = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [23]:
def tokenize(examples):
    return tokenizer(examples['message'], truncation=True)

train_ds = ds['train'].map(
    tokenize, 
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
eval_ds = ds['validation'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
train_ds.to_pandas()

Map:   0%|          | 0/96891 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

,label,input_ids,attention_mask
0,1,"[0, 9232, 2440, 12901, 1301, 27785, 14084, 662...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,"[0, 12, 3358, 1137, 162, 220, 86, 14, 13103, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1,"[0, 257, 236, 10, 10178, 50, 402, 116, 741, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,0,"[0, 271, 8494, 8271, 842, 2084, 2489, 13561, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[0, 417, 2533, 1920, 748, 34090, 912, 562, 309...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
96886,1,"[0, 9839, 98, 117, 2413, 1423, 118, 4710, 1423...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
96887,1,"[0, 10010, 33, 117, 12802, 506, 1371, 41166, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
96888,0,"[0, 11018, 39750, 4709, 7586, 24151, 784, 7842...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
96889,1,"[0, 10932, 1437, 2084, 242, 1437, 47, 393, 821...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


Create the transformer model:

In [24]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [25]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    transformer_name,
    num_labels=len(labels)
)

model = (
    BertForSequenceClassification
    .from_pretrained(transformer_name, config=config).to(device)
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerN

Create the trainer object and train:

In [26]:
from transformers import TrainingArguments

num_epochs = 2
batch_size = 48
weight_decay = 0.01
model_name = f'{transformer_name}-sequence-classification'

training_args = TrainingArguments(
    output_dir=model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    weight_decay=weight_decay,
    fp16=True
)

In [27]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [28]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
)

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.243100,0.340067,0.855670
2,0.169600,0.165737,0.938144


TrainOutput(global_step=4038, training_loss=0.25934077950619777, metrics={'train_runtime': 5634.8588, 'train_samples_per_second': 34.39, 'train_steps_per_second': 0.717, 'total_flos': 3.657850212431034e+16, 'train_loss': 0.25934077950619777, 'epoch': 2.0})

In [30]:
trainer.save_model("league_cyberb_dota-ROBERTA")

In [32]:
dota_test

,message,label
0,that costs 0 mana,0
1,pl wanna chase me asshole,1
2,CHE YL'T,0
3,YOU fuck only your mother,1
4,Report monkey,0
...,...,...
3161,have to hunt am at the beginning,0
3162,CK,0
3163,this team even more retarded,1
3164,REPORT MIRANA,0


Evaluate on the test partition:

In [23]:
output = trainer.predict(test_ds)
output

PredictionOutput(predictions=array([[ 3.421875  , -3.234375  ],
       [-1.3349609 ,  0.9951172 ],
       [-0.39624023,  0.2421875 ],
       ...,
       [ 0.9892578 , -0.9477539 ],
       [-4.2460938 ,  3.2753906 ],
       [-4.4453125 ,  3.4160156 ]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 1]), metrics={'test_loss': 0.20473513007164001, 'test_accuracy': 0.9100010320982558, 'test_runtime': 80.2246, 'test_samples_per_second': 120.773, 'test_steps_per_second': 2.518})

In [24]:
from sklearn.metrics import classification_report

y_true = output.label_ids
y_pred = np.argmax(output.predictions, axis=-1)
target_names = ["not toxic", "toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   not toxic       0.89      0.91      0.90      4424
       toxic       0.93      0.91      0.92      5265

    accuracy                           0.91      9689
   macro avg       0.91      0.91      0.91      9689
weighted avg       0.91      0.91      0.91      9689



In [33]:
dota_ds = DatasetDict()
dota_ds['test'] = Dataset.from_pandas(dota_test)
dota_ds

DatasetDict({
    test: Dataset({
        features: ['message', 'label'],
        num_rows: 3166
    })
})

In [34]:
test_ds_dota = dota_ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds_dota.to_pandas()

Map:   0%|          | 0/3166 [00:00<?, ? examples/s]

,label,input_ids,attention_mask
0,0,"[0, 6025, 1042, 321, 46339, 2]","[1, 1, 1, 1, 1, 1]"
1,1,"[0, 2911, 23126, 7859, 162, 43870, 2]","[1, 1, 1, 1, 1, 1, 1]"
2,0,"[0, 3764, 717, 854, 574, 108, 565, 2]","[1, 1, 1, 1, 1, 1, 1, 1]"
3,1,"[0, 38158, 26536, 129, 110, 985, 2]","[1, 1, 1, 1, 1, 1, 1]"
4,0,"[0, 20697, 25684, 2]","[1, 1, 1, 1]"
...,...,...,...
3161,0,"[0, 11990, 7, 8131, 524, 23, 5, 1786, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
3162,0,"[0, 347, 530, 2]","[1, 1, 1, 1]"
3163,1,"[0, 9226, 165, 190, 55, 47304, 2]","[1, 1, 1, 1, 1, 1, 1]"
3164,0,"[0, 4629, 20275, 256, 5216, 14629, 2]","[1, 1, 1, 1, 1, 1, 1]"


In [35]:
output_dota = trainer.predict(test_ds_dota)
output_dota

PredictionOutput(predictions=array([[-0.36523438,  0.9848633 ],
       [-2.3378906 ,  2.3945312 ],
       [ 1.296875  , -0.38500977],
       ...,
       [-3.15625   ,  2.859375  ],
       [ 1.8369141 , -0.85253906],
       [-3.9160156 ,  3.234375  ]], dtype=float32), label_ids=array([0, 1, 0, ..., 1, 0, 0]), metrics={'test_loss': 1.3961697816848755, 'test_accuracy': 0.6051800379027164, 'test_runtime': 1.6825, 'test_samples_per_second': 1881.69, 'test_steps_per_second': 39.227})

In [38]:
from sklearn.metrics import classification_report

y_true = output_dota.label_ids
y_pred = np.argmax(output_dota.predictions, axis=-1)
target_names = ["not toxic", "toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   not toxic       0.66      0.58      0.62      1736
       toxic       0.55      0.64      0.59      1430

    accuracy                           0.61      3166
   macro avg       0.61      0.61      0.60      3166
weighted avg       0.61      0.61      0.61      3166



## Testing for fun

In [32]:
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import Trainer
from transformers import AutoConfig
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

from transformers import AutoTokenizer

transformer_name = "league_model"
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')


class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

config = AutoConfig.from_pretrained(
    "league_model",
    #num_labels=len(labels),
    num_labels=2
)

model = (
    BertForSequenceClassification
    .from_pretrained("league_model", config=config)
)

trainer = Trainer(
    model=model,
)

In [40]:
cols = ["text", "label"]
data = [["soup",0],
        ["this is a soup", 0],
        ["in the library", 0],
        ["Motherfucking noob",1],
        ["AHAHHHAHHASDHADHU",0],
        ["bread and butter",0],
        ["soup is bad",0],
        ["I hope you beat your children",1],
        ["die soup",0], 
        ["I hope you die",1],
        ["I hope your kids die",1],
        ["I hope your kids", 0],
        ["I hope your kids soup", 0],
        ["die udyr", 1],
        ["die kids", 1],
        ["I hope your lose",0],
        ["I hope your succeed",0],
       ]
test = pd.DataFrame(data, columns = cols)

# Convert the DataFrame to a Dataset
test_dataset = Dataset.from_pandas(test)
#test_dataset.reset_index(inplace=True, drop=True)

dota_ds=DatasetDict()
dota_ds['test'] = Dataset.from_pandas(test)
#dota_ds.reset_index(inplace=True, drop=True)


# Now, map the tokenization function to the dataset
# Since your key for the dataset is 'test', make sure you access 'test' from dota_ds
test_ds_dota = dota_ds['test'].map(
    tokenize,
    batched=True,
    # remove_columns=['message'],  # Removing 'message' column after tokenization
)

# Convert to pandas DataFrame (for inspection)
test_ds_dota_df = test_ds_dota.to_pandas()

# Output the tokenized dataset
# print(test_ds_dota_df)


# Make predictions
output_dota = trainer.predict(test_ds_dota)

# Print or inspect the output
# print(output_dota)

output_dota.predictions

chats = test_dataset["text"]
labels = np.argmax(output_dota.predictions, axis=-1)
pd.DataFrame(labels, chats)

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

,0
soup,0
this is a soup,1
in the library,1
Motherfucking noob,1
AHAHHHAHHASDHADHU,0
bread and butter,1
soup is bad,1
I hope you beat your children,1
die soup,0
I hope you die,0
